# Выполнил Кирилл Сетдеков

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from statsmodels.stats.weightstats import *
import matplotlib.pyplot as plt
from statsmodels.stats.descriptivestats import sign_test

import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer
import seaborn as sns
from sklearn.preprocessing import PowerTransformer


import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.dates as mdates
from pandas.plotting import lag_plot

In [29]:
%matplotlib widget

#  Задание 1

1. Автокорреляционная функция и частная автокорреляционная функция.

    * Автокорреляционная функция - функция, которая для каждого значения лага выдает коэффициент корреляции между временным рядом и временным рядом с эти лагом.
    * Частная автокорреляционная функция - функция, которая для каждого лага k выдает автокорреляцию между исходным рядом $z_{t}$, очиценным от линейной связи со всеми предшествующими лагами от $z_{t+1}$ до $z_{t+k-1}$ и рядом  $z_{{t+k}}$. 

2. Какой временной ряд называется стационарным? Может ли временной ряд с трендом быть стационарным? А с циклом?

    * Стационарный ряд - такой ряд, у которого вероятностные характеристики не меняются во времени.

    * Если говорим про тренд - мы будем ожидать, что такой ряд интегрированный порядка 1, и сам не стационарный. Но ряд из растностей может быть стационарным, а может и не быть.

    * Если мы говорим про циклы, имеющие некий фиксированный период (сезонность), то такой ряд не стационарный, если длина цикла случайна, то такой ряд может быть стационарным.
  

3. Какую гипотезу проверяет тест Дики-Фуллера.

    * Тесть проверяет гипотезу о стационарности ряда и $H_0$ - ряд не стационарный, то есть когда мы отвергаем гипотезу, мы узнаем, что ряд стационарный.

4. Какой временной рядм можно описать хорошо с помощью модели ARMA?

    * Под эту модель подходят хорошо ряды со следующими свойствами:

        * Ряд стационарный.
        * Нет внешних переменных, которые влияют на значения ряда (сходу без модификации в эту модель не добавить регрессую от других рядов).
        * Есть зависимость от значений переменной в прошлом и от остатков на прошлых шагах.

5. На что нужно обращать внимание при анализе остатков модели

    * Проверять соответствие их распределения нормальному.
    * Тестировать их на стационарность.
    * Обращать внимание на наличие периодичности и/или гетероскедастичности.



# Задание 2
Шампунь, продажи за 3 года. Подобрать модель SARIMA и сделать прогноз на 3 месяца вперед.

In [30]:
shampoo = pd.read_csv('data/shampoo.csv')


In [31]:
shampoo["Month"] = pd.to_datetime('0'+shampoo.Month, format='%y-%m')
shampoo = shampoo.set_index("Month")
shampoo.head()

,Sales
Month,
2001-01-01,266.0
2001-02-01,145.9
2001-03-01,183.1
2001-04-01,119.3
2001-05-01,180.3


In [32]:
decompostition = sm.tsa.seasonal_decompose(shampoo, model= 'additive')
fig = decompostition.plot()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
plt.figure()
lag_plot(shampoo)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Видим, что есть тренд, и сезонность, ожидание, что ряд не стационарный и что есть автокорреляция. Проверим:

In [34]:
from statsmodels.tsa.stattools import adfuller


def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [35]:
print("Критерий Дики-Фуллера для начального ряда: pvalue =",sm.tsa.stattools.adfuller(shampoo.Sales)[1])

Критерий Дики-Фуллера для начального ряда: pvalue = 1.0


In [36]:
adf_test(shampoo)

Results of Dickey-Fuller Test:
Test Statistic                  3.060142
p-value                         1.000000
#Lags Used                     10.000000
Number of Observations Used    25.000000
Critical Value (1%)            -3.723863
Critical Value (5%)            -2.986489
Critical Value (10%)           -2.632800
dtype: float64


Такой ряд не стационарный - перейдем к разностям:

In [37]:
shampoo['sales_boxcox'], lmbd = stats.boxcox(shampoo['Sales'])
shampoo['sales_boxcox_diff'] = shampoo.sales_boxcox - shampoo.sales_boxcox.shift(1)
sm.tsa.seasonal_decompose(shampoo.sales_boxcox_diff[1:]).plot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
adf_test(shampoo.sales_boxcox_diff[1:])

Results of Dickey-Fuller Test:
Test Statistic                 -3.971330
p-value                         0.001568
#Lags Used                      9.000000
Number of Observations Used    25.000000
Critical Value (1%)            -3.723863
Critical Value (5%)            -2.986489
Critical Value (10%)           -2.632800
dtype: float64


In [39]:
shampoo['sales_diff'] = shampoo.Sales - shampoo.Sales.shift(1)
sm.tsa.seasonal_decompose(shampoo.sales_diff[1:]).plot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
adf_test(shampoo.sales_diff[1:])

Results of Dickey-Fuller Test:
Test Statistic                -7.249074e+00
p-value                        1.799857e-10
#Lags Used                     1.000000e+00
Number of Observations Used    3.300000e+01
Critical Value (1%)           -3.646135e+00
Critical Value (5%)           -2.954127e+00
Critical Value (10%)          -2.615968e+00
dtype: float64


без преобразования Бокса-Кокса, мы получаем более низкое p-value для теста Дики-Фулера, поэтому возьмем только разницы, а чтобы брать дельты дополнительные, уже нет данных

посмотрим на автокорреляционные и частные автокорреляционные графики

In [41]:
fig, (ax1, ax2) = plt.subplots(nrows=2,figsize=(15,8)) 

sm.graphics.tsa.plot_acf(shampoo.sales_diff[1:].values.squeeze(), lags=15, ax=ax1)
sm.graphics.tsa.plot_pacf(shampoo.sales_diff[1:].values.squeeze(), lags=15, ax=ax2)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Согласно этим графикам, в качестве начальных значений необходимо взять: $Q=1$, $q=1$, $P=1$, $p=1$.

In [42]:
from itertools import product # произведения множеств

# создадим список из значений параметров для перебора моделей


d = 1
D = range(0, 2)
ps = range(0, 2)
qs = range(0, 2)
Ps = range(0, 2)
Qs = range(0, 2)
trend = ['n', 'c', 't', 'ct']

parameters = product(ps ,qs, Ps, Qs, D, trend)
parameters_list = list(parameters)

In [43]:
from tqdm.notebook import tqdm


In [44]:
results = [] # здесь будем хранить результаты
warnings.filterwarnings('ignore') # не для всех параметров модели будут обучаться; отключим предупреждения

for param in tqdm(parameters_list):
    #try-except нужен, чтобы обработать случаи, когда модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(shampoo.Sales, order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], param[4], param[3], 12), trend=param[5]).fit(disp=-1)
    
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError:
        print('Wrong parameters:', param)
        continue
    
    results.append([param, model.aic])
# преобразуем результаты в DataFrame и добавим названия столбцов

result_table = pd.DataFrame(results)
result_table.columns = ['Parameters', 'AIC']

  0%|          | 0/128 [00:00<?, ?it/s]

In [45]:
result_table.sort_values(by = 'AIC', ascending=True).head()

,Parameters,AIC
117,"(1, 1, 1, 0, 1, c)",275.535467
109,"(1, 1, 0, 1, 1, c)",275.935619
118,"(1, 1, 1, 0, 1, t)",276.586929
119,"(1, 1, 1, 0, 1, ct)",277.122648
125,"(1, 1, 1, 1, 1, c)",277.258594


Лучшей моделью получилась SARIMA (1,1,1)x(1,0,1).

In [46]:
best_model=sm.tsa.statespace.SARIMAX(shampoo.Sales, order=(1,1,1), trend='c', seasonal_order=(1,1,0,12)).fit(disp=-1)
print(best_model.summary())

                                      SARIMAX Results                                      
Dep. Variable:                               Sales   No. Observations:                   36
Model:             SARIMAX(1, 1, 1)x(1, 1, [], 12)   Log Likelihood                -132.768
Date:                             Thu, 05 Aug 2021   AIC                            275.535
Time:                                     23:43:47   BIC                            281.213
Sample:                                 01-01-2001   HQIC                           276.963
                                      - 12-01-2003                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     27.5420      4.660      5.910      0.000      18.408      36.676
ar.L1         -0.6273      

Посмотрим на остатки

In [47]:
best_model.resid[13:].plot()
plt.ylabel('Residuals')
plt.show()

**Несмещенность.** Проверим несмещенность с помощью критерия знаков (самый универсальный критерий).

In [48]:
from statsmodels.stats.descriptivestats import sign_test
print("Критерий знаков для остатков: pvalue =",sign_test(best_model.resid[13:])[1])

Критерий знаков для остатков: pvalue = 0.21003961563110352


**Стационарность.** Проверим стационарность с помщью критерия Дики-Фуллера.

In [49]:
print("Критерий Дики-Фуллера для остатков: pvalue =",sm.tsa.stattools.adfuller(best_model.resid[13:])[1])

Критерий Дики-Фуллера для остатков: pvalue = 0.000610127398725332


**Неавтокоррелированность.** Чтобы определить независимость остаткок, посмотрим на график автокорреляционной функции.

In [50]:
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=15)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Еще и по Jarque-Bera не отвергается гипотеза о нормальности

In [51]:
figure = best_model.plot_diagnostics()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
shampoo['model'] = best_model.fittedvalues

In [53]:
fig, ax = plt.subplots()
ax.plot(shampoo.Sales, label='actual')
ax.plot(shampoo.model, color='r', label='prediction')
ax.plot(best_model.get_forecast(12).predicted_mean, label='forecast')
ax.plot(best_model.get_forecast(12).conf_int()['lower Sales'], label='lower_conf', color = 'gray')
ax.plot(best_model.get_forecast(12).conf_int()['upper Sales'], label='upper_conf', color = 'gray')
ax.fill_between(best_model.get_forecast(12).predicted_mean.index, 
                best_model.get_forecast(12).conf_int()['lower Sales'], 
                best_model.get_forecast(12).conf_int()['upper Sales'], facecolor='lightgray', alpha=0.5)
ax.legend()

ax.set_ylabel('volume')
ax.set_title('Shampoo sales')
fig.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Задание 3

